In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from pyspark.sql.types import StructType, StructField, StringType, BinaryType, IntegerType, DoubleType, TimestampType, DateType, LongType
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
from pyspark.storagelevel import StorageLevel
from typing import Union, Optional
from pyspark.sql.functions import input_file_name

# --- Credenciais AWS ---
accessKeyId = ""
secretAccessKey = ""

# --- Sessão Spark ---
def create_spark_session() -> SparkSession:
    spark = (
        SparkSession
        .builder
        .appName("Bronze Zone Streaming")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .enableHiveSupport()
        .getOrCreate()
    )
    
    spark.sparkContext.setLogLevel("WARN")

    conf = spark.sparkContext._jsc.hadoopConfiguration()
    conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
    conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    conf.set("fs.s3a.fast.upload", "true")
    conf.set("fs.s3a.bucket.all.committer.magic.enabled", "true")
    conf.set("fs.s3a.directory.marker.retention", "keep")
    conf.set("spark.driver.extraClassPath", "/usr/local/spark/jars/*")
    conf.set("spark.driver.memory", "8g")
    conf.set("spark.executor.memory", "16g")
    conf.set("fs.s3a.access.key", accessKeyId)
    conf.set("fs.s3a.secret.key", secretAccessKey)

    return spark

spark = create_spark_session()

In [4]:
bronze_path = f"s3a://dev-lab-02-us-east-2-bronze/spotify/"
spark.read.format("delta").load(f"{bronze_path}usuarios").createOrReplaceTempView("usuarios")

In [15]:
spark.sql(
"""
select count(1) from usuarios
"""
).show()

+--------+
|count(1)|
+--------+
|     110|
+--------+



In [14]:
spark.sql(
"""
select * from usuarios
order by timestamp desc
"""
).show()

+---+--------------------+--------------------+-------------------+--------------------+
| id|                nome|               email|          timestamp|      origem_arquivo|
+---+--------------------+--------------------+-------------------+--------------------+
| 22|  Dr. Paulette Aubry|virginie60@exampl...|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 42|       Marvin Staude|siegmar34@example...|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 23|      Jaroslaw Karge|xschlosser@exampl...|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 27| Benigno Ocaña López|victoriagomila@ex...|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 28|  Karl-Heinrich Henk|enginkambs@exampl...|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 11|  Micaela Jara Peiró| cosme82@example.net|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 29|   Sylvie Carpentier| hbrunel@example.net|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 13|        Émilie Baron|charlescapucine@e...|2025-05-06 15:01:20|s3a://dev-lab-02-...|
| 30|   Gabrielly Rib